# Day 9 - Part 1

In [ ]:
with open("red_tiles.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

big = 0
for i in range(len(data)):
    for y in range(i + 1, len(data)):
        dx = abs(data[i][0] - data[y][0])
        dy = abs(data[i][1] - data[y][1])
        area = dx * dy
        if area > big:
            big = area
            p1, p2 = data[i], data[y]

print(big)

4754817236


# Day 9 - Part 2

In [ ]:
import numpy as np

with open("red_tiles_t.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

max_x, max_y = max(x for x, _ in data), max(y for _, y in data)
min_x, min_y = min(x for x, _ in data), min(y for _, y in data)

data = np.array(data)

top = data[data[:,1] == min_y]
bottom = data[data[:,1] == max_y]
left = data[data[:,0] == min_x]
right = data[data[:,0] == max_x]

min_x_top,    max_x_top    = top[:,0].min(),    top[:,0].max()
min_x_bottom, max_x_bottom = bottom[:,0].min(), bottom[:,0].max()
min_y_left,   max_y_left   = left[:,1].min(),   left[:,1].max()
min_y_right,  max_y_right  = right[:,1].min(),  right[:,1].max()

top_inside = np.array([(x, min_y) for x in range(top[:,0].min() + 1, top[:,0].max())])
bottom_inside = np.array([(x, max_y) for x in range(bottom[:,0].min() + 1, bottom[:,0].max())])
left_inside = np.array([(min_x, y) for y in range(left[:,1].min() + 1, left[:,1].max())])
right_inside = np.array([(max_x, y) for y in range(right[:,1].min() + 1, right[:,1].max())])

c_sides = np.vstack((top, bottom, left, right))
sides_inside = np.vstack((top_inside, bottom_inside, left_inside, right_inside))

# grid setup
grid = np.full((max_y + 1, max_x + 1), '.', dtype='<U1')

corner_mask = np.zeros_like(grid, dtype=bool)
corner_mask[c_sides[:,1], c_sides[:,0]] = True

line_mask = np.zeros_like(grid, dtype=bool)
line_mask[sides_inside[:,1], sides_inside[:,0]] = True

infill_mask = np.zeros_like(grid, dtype=bool)
infill_mask[min_y_right+1:max_y_right, max(min_x_bottom,min_x_top)+1:max_x] = True
infill_mask[min_y+1:min(max_y_left,max_y_right), min_x_top+1:max_x_top] = True
infill_mask[min_y_left+1:max_y_left, min_x+1:min(min_x_bottom,min_x_top)] = True
infill_mask[max(min_y_left,min_y_right)+1:max_y, min_x_bottom+1:max_x_bottom] = True

grid[corner_mask] = '#'
grid[line_mask]   = 'X'
grid[infill_mask] = 'X'

print(top)
print(grid)

big = 0
for i in range(len(c_sides)):
    for y in range(i + 1, len(c_sides)):
        dx = abs(c_sides[i,0] - c_sides[y,0]) + 1
        dy = abs(c_sides[i,1] - c_sides[y,1]) + 1
        area = dx * dy
        if area > big:
            big = area
            p1, p2 = c_sides[i], c_sides[y]

print(big)




[[ 7  1]
 [11  1]]
[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' 'X' 'X' 'X' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' 'X' 'X' 'X' 'X']
 ['.' '.' '#' '.' '.' '.' '.' '.' 'X' 'X' 'X' 'X']
 ['.' '.' 'X' 'X' 'X' 'X' 'X' '.' 'X' 'X' 'X' 'X']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '.' 'X' 'X']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' 'X' 'X']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#' 'X' '#']]
50


In [ ]:
with open("red_tiles_t.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

max_x, max_y = max(x for x, _ in data), max(y for _, y in data)
min_x, min_y = min(x for x, _ in data), min(y for _, y in data)

top    = [p for p in data if p[1] == min_y]
bottom = [p for p in data if p[1] == max_y]
left   = [p for p in data if p[0] == min_x]
right  = [p for p in data if p[0] == max_x]

c_sides = top + bottom + left + right

y_top    = min_y
y_bottom = max_y
x_right  = max_x
x_left   = min_x

max_x_top,    max_y_top    = max(x for x, _ in top),    max(y for _, y in top)
max_x_bottom, max_y_bottom = max(x for x, _ in bottom), max(y for _, y in bottom)
max_x_left,   max_y_left   = max(x for x, _ in left),   max(y for _, y in left)
max_x_right,  max_y_right  = max(x for x, _ in right),  max(y for _, y in right)

min_x_top,    min_y_top    = min(x for x, _ in top),    min(y for _, y in top)
min_x_bottom, min_y_bottom = min(x for x, _ in bottom), min(y for _, y in bottom)
min_x_left,   min_y_left   = min(x for x, _ in left),   min(y for _, y in left)
min_x_right,  min_y_right  = min(x for x, _ in right),  min(y for _, y in right)

top_range    = {'x': range(min_x_top, max_x_top + 1), 'y': range(y_top, min(max_y_left, max_y_right) + 1)}
bottom_range = {'x': range(min_x_bottom, max_x_bottom + 1), 'y': range(max(min_y_left,  min_y_right), y_bottom + 1)}
left_range   = {'x': range(min_x_left, min(max_x_bottom, max_x_top) + 1), 'y': range(min_y_left, max_y_left + 1)}
right_range  = {'x': range(max(min_x_bottom, min_x_top), x_right + 1), 'y': range(min_y_right, max_y_right + 1)}

b_ranges = [top_range, bottom_range, left_range, right_range]

box_top    = [(x, y) for x in range(min_x_top, max_x_top + 1) for y in range(y_top, min(max_y_left, max_y_right) + 1)]
box_bottom = [(x, y) for x in range(min_x_bottom, max_x_bottom + 1) for y in range(max(min_y_left,  min_y_right), y_bottom + 1)]
box_left   = [(x, y) for x in range(min_x_left, min(max_x_bottom, max_x_top) + 1) for y in range(min_y_left, max_y_left + 1)]
box_right  = [(x, y) for x in range(max(min_x_bottom, min_x_top), x_right + 1) for y in range(min_y_right, max_y_right + 1)]

edges = set(box_top + box_bottom + box_left + box_right)

grid = [['.' for _ in range(max_x + 1)] for _ in range(max_y + 1)]

for row, col in c_sides:
    grid[col][row] = '#'

for row, col in edges:
    grid[col][row] = 'X' if grid[col][row] != '#' else '#'

for line in grid:
    print(line)

def check_bounding(p1, p2):
    x1, y1 = p1
    x2, y2 = p2

    for br in b_ranges:
        if (x1 in br['x'] and y1 in br['y']) and (x2 in br['x'] and y2 in br['y']):
            return True

    return False

big = 0
for i in range(len(data)):
    for y in range(i + 1, len(data)):
        if not (check_bounding(data[i], data[y])):
            continue

        dx = abs(data[i][0] - data[y][0]) + 1
        dy = abs(data[i][1] - data[y][1]) + 1
        area = dx * dy

        if area > big:
            big = area
            p1, p2 = data[i], data[y]

print(big)

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '#', 'X', 'X', 'X', '#']
['.', '.', '.', '.', '.', '.', '.', 'X', 'X', 'X', 'X', 'X']
['.', '.', '#', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['.', '.', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['.', '.', '#', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['.', '.', '.', '.', '.', '.', '.', '.', '.', 'X', 'X', 'X']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '#', 'X', '#']
24


In [ ]:
with open("red_tiles_t.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

max_x, max_y = max(x for x, _ in data), max(y for _, y in data)
min_x, min_y = min(x for x, _ in data), min(y for _, y in data)

top    = [p for p in data if p[1] == min_y]
bottom = [p for p in data if p[1] == max_y]
left   = [p for p in data if p[0] == min_x]
right  = [p for p in data if p[0] == max_x]

y_top    = min_y
y_bottom = max_y
x_right  = max_x
x_left   = min_x

max_x_top,    max_y_top    = max(x for x, _ in top),    max(y for _, y in top)
max_x_bottom, max_y_bottom = max(x for x, _ in bottom), max(y for _, y in bottom)
max_x_left,   max_y_left   = max(x for x, _ in left),   max(y for _, y in left)
max_x_right,  max_y_right  = max(x for x, _ in right),  max(y for _, y in right)

min_x_top,    min_y_top    = min(x for x, _ in top),    min(y for _, y in top)
min_x_bottom, min_y_bottom = min(x for x, _ in bottom), min(y for _, y in bottom)
min_x_left,   min_y_left   = min(x for x, _ in left),   min(y for _, y in left)
min_x_right,  min_y_right  = min(x for x, _ in right),  min(y for _, y in right)

top_range    = {'x': range(min_x_top, max_x_top + 1),                     'y': range(y_top, min(max_y_left, max_y_right) + 1)}
bottom_range = {'x': range(min_x_bottom, max_x_bottom + 1),               'y': range(max(min_y_left,  min_y_right), y_bottom + 1)}
left_range   = {'x': range(min_x_left, min(max_x_bottom, max_x_top) + 1), 'y': range(min_y_left, max_y_left + 1)}
right_range  = {'x': range(max(min_x_bottom, min_x_top), x_right + 1),    'y': range(min_y_right, max_y_right + 1)}

b_x_ranges = [top_range['x'], bottom_range['x'], left_range['x'], right_range['x']]
b_y_ranges = [top_range['y'], bottom_range['y'], left_range['y'], right_range['y']]

top_bounds    = {'x': (min_x_top, max_x_top),                     'y': (y_top, min(max_y_left, max_y_right))}
bottom_bounds = {'x': (min_x_bottom, max_x_bottom),               'y': (max(min_y_left, min_y_right), y_bottom)}
left_bounds   = {'x': (min_x_left, min(max_x_bottom, max_x_top)), 'y': (min_y_left, max_y_left)}
right_bounds  = {'x': (max(min_x_bottom, min_x_top), x_right),    'y': (min_y_right, max_y_right)}

b_x_ranges = [top_bounds['x'], bottom_bounds['x'], left_bounds['x'], right_bounds['x']]
b_y_ranges = [top_bounds['y'], bottom_bounds['y'], left_bounds['y'], right_bounds['y']]

# slice the box by bounding boxes, make new boxes, if empty at the end == fits in shape
def check_bounding(p1, p2):
    x1, y1 = p1
    x2, y2 = p2

    x_min, x_max = min(x1, x2), max(x1, x2)
    y_min, y_max = min(y1, y2), max(y1, y2)

    p1 = (x_min, y_min)
    p2 = (x_max, y_max)

    points = [(p1, p2)]

    for xr in b_x_ranges:
        x_max = x_max - xr[0]
        x_min = x_min - xr[1]

    for yr in b_y_ranges:
        if y_min < yr[0] or y_max > yr[1]:
            return False

    return True


big = 0
for i in range(len(data)):
    for y in range(i + 1, len(data)):
        if not (check_bounding(data[i], data[y])):
            continue

        dx = abs(data[i][0] - data[y][0]) + 1
        dy = abs(data[i][1] - data[y][1]) + 1
        area = dx * dy

        if area > big:
            big = area
            p1, p2 = data[i], data[y]

print(big)

# Final Solution

In [ ]:
with open("red_tiles.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

max_x, max_y = max(x for x, _ in data), max(y for _, y in data)
min_x, min_y = min(x for x, _ in data), min(y for _, y in data)

top    = [p for p in data if p[1] == min_y]
bottom = [p for p in data if p[1] == max_y]
left   = [p for p in data if p[0] == min_x]
right  = [p for p in data if p[0] == max_x]

y_top    = min_y
y_bottom = max_y
x_right  = max_x
x_left   = min_x

max_x_top,    max_y_top    = max(x for x, _ in top),    max(y for _, y in top)
max_x_bottom, max_y_bottom = max(x for x, _ in bottom), max(y for _, y in bottom)
max_x_left,   max_y_left   = max(x for x, _ in left),   max(y for _, y in left)
max_x_right,  max_y_right  = max(x for x, _ in right),  max(y for _, y in right)

min_x_top,    min_y_top    = min(x for x, _ in top),    min(y for _, y in top)
min_x_bottom, min_y_bottom = min(x for x, _ in bottom), min(y for _, y in bottom)
min_x_left,   min_y_left   = min(x for x, _ in left),   min(y for _, y in left)
min_x_right,  min_y_right  = min(x for x, _ in right),  min(y for _, y in right)

top_bounds    = {'x': (min_x_top, max_x_top),                     'y': (y_top, min(max_y_left, max_y_right))}
bottom_bounds = {'x': (min_x_bottom, max_x_bottom),               'y': (max(min_y_left, min_y_right), y_bottom)}
left_bounds   = {'x': (min_x_left, min(max_x_bottom, max_x_top)), 'y': (min_y_left, max_y_left)}
right_bounds  = {'x': (max(min_x_bottom, min_x_top), x_right),    'y': (min_y_right, max_y_right)}

b_x_ranges = [top_bounds['x'], bottom_bounds['x'], left_bounds['x'], right_bounds['x']]
b_y_ranges = [top_bounds['y'], bottom_bounds['y'], left_bounds['y'], right_bounds['y']]

# slice the box by bounds, make new boxes, if empty at the end == fits in shape
def check_bounding(p1, p2):
    x1, y1 = p1
    x2, y2 = p2

    x_min, x_max = min(x1, x2), max(x1, x2)
    y_min, y_max = min(y1, y2), max(y1, y2)

    xp = [(x_min, x_max)]
    yp = [(y_min, y_max)]

    for xr in b_x_ranges:
        new_p_list = []

        for p in xp:
            left = (xr[0], p[0])
            right = (xr[1], p[1])

            if left[0] < left[1]:
                new_p_list.append(left)
            if right[0] < right[1]:
                new_p_list.append(right)

        xp = new_p_list[:]
        if not xp:
            break

    for yr in b_y_ranges:
        new_p_list = []

        for p in yp:
            top = (yr[0], p[0])
            bottom = (yr[1], p[1])

            if top[0] < top[1]:
                new_p_list.append(top)
            if bottom[0] < bottom[1]:
                new_p_list.append(bottom)

        yp = new_p_list[:]
        if not yp:
            break

    if xp or yp:
        return False

    return True


big = 0
for i in range(len(data)):
    for y in range(i + 1, len(data)):
        if not (check_bounding(data[i], data[y])):
            continue

        dx = abs(data[i][0] - data[y][0]) + 1
        dy = abs(data[i][1] - data[y][1]) + 1
        area = dx * dy

        if area > big:
            big = area
            p1, p2 = data[i], data[y]

print((p1, p2), big)

((84879, 84065), (14147, 16842)) 4754955192


In [ ]:
from dataclasses import dataclass


@dataclass
class Point:
    x: int
    y: int


class Rect:
    def __init__(self, p1: Point, p2: Point):
        # normalize so origin is always top right
        self.left   = min(p1.x, p2.x)
        self.right  = max(p1.x, p2.x)
        self.top    = min(p1.y, p2.y)
        self.bottom = max(p1.y, p2.y)

    def get_area(self):
        return (self.get_width() + 1) * (self.get_height() + 1)  # inclusive
    
    def get_width(self):
        return self.right - self.left
    
    def get_height(self):
        return self.bottom - self.top
    
    def get_origin(self):
        return Point(self.left, self.top)
    
    def get_points(self):
        # points in clockwise order
        self.top_left     = Point(self.left, self.top)
        self.top_right    = Point(self.right, self.top)
        self.bottom_right = Point(self.right, self.bottom)
        self.bottom_left  = Point(self.left, self.bottom)

        return self.top_left, self.top_right, self.bottom_right, self.bottom_left

    def __repr__(self):
        return f"Rect(P1({self.left},{self.top}) P2({self.right},{self.bottom}))"


def overlap_r(r1: Rect, r2: Rect):
    overlap_x = r1.left <= r2.right and r1.right >= r2.left
    overlap_y = r1.top <= r2.bottom and r1.bottom >= r2.top

    x, y = None, None

    if overlap_x:
        x = max(r1.left, r2.left) 

    if overlap_y:
        y = max(r1.top, r2.top)

    if overlap_x and overlap_y:
        return Rect(Point(x, y), Point(min(r1.right, r2.right), min(r1.bottom, r2.bottom)))

    return False


# cuts horizontaly (left to right)
def cut_h_segments(slab: Rect, cutter: Rect):
    ov = overlap_r(slab, cutter)
    cuts = []

    if not ov:
        return None
    
    if ov.left > slab.left:
        cuts.append(Rect(Point(slab.left, ov.top), ov.bottom_left))

    if ov.right < slab.right:
        cuts.append(Rect(ov.top_right, Point(slab.right, ov.bottom)))

    if ov.top > slab.top:
        cuts.append(Rect(slab.top_left, Point(slab.right, ov.top)))

    if ov.bottom < slab.bottom:
        cuts.append(Rect(Point(slab.left, ov.bottom), slab.bottom_right))

    return cuts


# TODO: implement - cuts vertically (top to bottom)
def cut_v_segments(slab: Rect, cutter: Rect):
    pass


# xy
p1 = (2, 1)
p2 = (7, 6)

p3 = (1, 4)
p4 = (5, 9)

points = [p1, p2, p3, p4]
min_x, max_x = min([x for x, _ in points]), max([x for x, _ in points])
min_y, max_y = min([y for _, y in points]), max([y for _, y in points])

grid = [['.' for _ in range(max_x + 1)] for _ in range(max_y + 1)]

pob = [Rect(Point(p1[0], p1[1]), Point(p2[0], p2[1])),
       Rect(Point(p3[0], p3[1]), Point(p4[0], p4[1]))]

# for p in pob:
#     for point in p.points:
#         grid[point.y][point.x] = '#'

# new = cut_h_segments(pob[1], pob[0])

# for p in new:
#     for point in p.points:
#         grid[point.y][point.x] = 'x'


# --- solution
with open("red_tiles.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

data = [Point(x, y) for x, y in data]
edges = [Rect(data[i], data[i+1]) for i in range(len(data[:-1]))]
edges.append(Rect(data[-1], data[0]))

def overlap_r(r1: Rect, r2: Rect):
    if r1.left > r2.right or r1.right < r2.left: return False
    if r1.top  > r2.bottom or r1.bottom < r2.top: return False
    return True

big = 0
for i in range(len(data)):
    for y in range(i + 1, len(data)):
        rect = Rect(data[i], data[y])
        has_overlap = any(overlap_r(rect, edge) for edge in edges)

        if not has_overlap:
            area = rect.get_area()
            if area > big:
                big = area
                last = rect

big

0